In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys 
sys.path.append("/Users/julianganzabal/facultad/lab-ml/mllab-tools")

In [403]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

In [263]:
model=get_deep_rnn((100, 69), dense_units=69)

input shape =  (100, 69)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 200)          216000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_2 (Dense)              (None, 69)                13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


In [584]:
# bodes_sangre_LSTM_deep
# bodes_sangre_LSTM_deep_dropout_04
model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')

In [585]:
def split_data_and_get_dicts(text, train_fraction=0.6):
    number_of_chars = len(text)
    validation_index = int(number_of_chars*train_fraction)
    text_train = text[:validation_index]
    text_validation = text[validation_index:]
    chars_train = set(text_train)
    chars_test = set(text_validation)
    chars_set = chars_train.intersection(chars_test)
    chars = sorted(list(chars_set))
    chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer
    indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character
    return text_train, text_validation, chars_to_indices, indices_to_chars, chars

In [586]:
text = open('bodas_de_sangre.txt').read()
text_train, text_validation, chars_to_indices, indices_to_chars, chars = split_data_and_get_dicts(text)

In [587]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 


In [588]:
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
print(X_test.shape)

(1, 100, 69)


In [589]:
probs = model.predict(X_test)
print(probs)
index = np.argmax(probs)
print(index)
print(indices_to_chars[index])

[[  2.67503987e-04   1.47814702e-04   6.57919809e-05   2.55768944e-04
    4.72745478e-06   2.37566437e-05   2.42656661e-05   2.75124034e-06
    3.70589163e-07   4.40745964e-08   3.66885047e-06   1.64531309e-06
    6.16733596e-05   7.41606073e-06   1.38893965e-05   8.09141711e-05
    5.61719680e-05   4.66866550e-05   1.43318912e-05   3.72988325e-06
    7.39485222e-06   2.96074631e-05   2.25158919e-06   1.05969375e-03
    2.02943091e-04   3.09547031e-05   3.36788726e-05   1.23046950e-04
    3.96312571e-05   1.16490854e-04   6.03278386e-05   4.03931081e-05
    1.16660331e-05   1.97234822e-05   2.18112382e-05   1.14414220e-08
    3.21773416e-03   1.22510139e-02   5.05333766e-02   1.11963920e-01
    7.49744996e-02   1.27892541e-02   1.42459823e-02   2.28233449e-02
    8.94579384e-03   8.82903347e-04   1.80775244e-02   6.82415813e-02
    1.20949820e-01   1.26354326e-03   6.70492575e-02   4.00202051e-02
    8.58152471e-03   8.35077390e-02   9.80713144e-02   1.13407243e-02
    1.37485445e-01  

## Muestrar modelo

In [590]:
temperatura = 1
sample([0.5, 0.25, 0.15, 0.10], temperatura, verbose=True)

[ 0.5   0.25  0.15  0.1 ]


3

In [591]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
for i in range(10):
    my_sample = sample(probs[0], 0.5)
    print(indices_to_chars[my_sample])

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 
v
v
t
d
v
d
d
e
v
v


In [592]:
window_size = 100
initial_char = 0
initial_text = text_validation[initial_char:window_size+initial_char]
print(initial_text)
X_test = chars_to_one_hot(initial_text, chars, chars_to_indices, window_size)
probs = model.predict(X_test)
for i in range(10):
    my_sample = sample(probs[0], 0.5)
    print(indices_to_chars[my_sample])

ovia. (Se va.)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novi
a
a
a
a
a
a
a
a
a
a


In [593]:
window_size = 100
initial_char = 16
initial_text = text_validation[initial_char:window_size+initial_char]

N = 200
X_text_str = initial_text
print(initial_text)
print('---------------------------------------------------')
print()
for i in range(N):
    X_test = chars_to_one_hot(X_text_str[i:], chars, chars_to_indices, window_size)
    probs = model.predict(X_test)
    my_sample=sample(probs[0], 0.5)
    new_char = indices_to_chars[my_sample]
    X_text_str = X_text_str + new_char
print(X_text_str)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 
---------------------------------------------------

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya verdes!

VECINA: Sí, pero de pendo a verdado no está me puede y el novio de lo mijo.

MADRE: ¿Y qué?

NOVIO: Me vanta esto y buena con la casa.

MADRE: ¡Vamos a la viña?

CRIADA: (Al novio) ¡Qué para 


- Mantiene estructuta de obra de teatro
- MADRE: ¿Y qué? -> Abre y cierra signo de pregunta 
- CRIADA: (Al novio) -> Abre y cierra parentesis
- el novio, la viña, la casa -> Articulo mas sustantivo con género